<a href="https://colab.research.google.com/github/ljkrajewski/jupyter_notebooks/blob/main/flux/png_meta_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install pre-reqs
!pip install gradio
!pip install requests
#!pip install exifread

In [ ]:
#@title Define functions
import gradio as gr
from PIL import Image
import os
import requests
from io import BytesIO

def find_best_fit_ratio(width, height):
    """
    Convert a resolution (width x height) to the best-fit ratio from a list of potential ratios.

    Args:
        width (int): Width of the resolution
        height (int): Height of the resolution
        potential_ratios (list of tuples): List of (width, height) ratio tuples, e.g., [(16, 9), (4, 3)]

    Returns:
        tuple: Best-fit ratio as (width, height), or None if inputs are invalid
    """
    potential_ratios = [(16, 9), (4, 3), (2,1), (1,1), (9,7)]
    if height > width:
        width, height = height, width
        invert = True
    else:
        invert = False
    try:
        # Validate inputs
        if not isinstance(width, (int, float)) or not isinstance(height, (int, float)):
            return None
        if width <= 0 or height <= 0:
            return None
        if not potential_ratios or not all(isinstance(r, tuple) and len(r) == 2 for r in potential_ratios):
            return None

        # Calculate input aspect ratio
        input_ratio = width / height

        # Find best-fit ratio by comparing differences in aspect ratios
        best_ratio = None
        min_difference = float('inf')

        for ratio in potential_ratios:
            ratio_width, ratio_height = ratio
            if ratio_width <= 0 or ratio_height <= 0:
                continue
            current_ratio = ratio_width / ratio_height
            difference = abs(input_ratio - current_ratio)

            if difference < min_difference:
                min_difference = difference
                best_ratio = (ratio_width, ratio_height)

        if invert:
            best_ratio = (best_ratio[1], best_ratio[0])

        return best_ratio

    except Exception:
        return None

def get_image_metadata(image, url):
    try:
        # Check if either image or URL is provided
        if image is None and not url:
            return "No image or URL provided"

        # Handle URL input
        if url:
            try:
                response = requests.get(url, timeout=10)
                response.raise_for_status()
                img = Image.open(BytesIO(response.content))
            except requests.RequestException as e:
                return f"Error fetching image from URL: {str(e)}"
            except Exception as e:
                return f"Error processing image from URL: {str(e)}"
        else:
            # Open image from file upload
            img = Image.open(image)

        # Find best-fit ratio
        best_fit_ratio = find_best_fit_ratio(img.size[0], img.size[1])

        # Initialize metadata dictionary
        metadata = {
            'Format': img.format,
            'Mode': img.mode,
            'Size': img.size,
            'Width': img.size[0],
            'Height': img.size[1],
            'Ratio': f"{best_fit_ratio[0]}:{best_fit_ratio[1]}"
        }

        # Get PNG-specific metadata from info dictionary
        if hasattr(img, 'info'):
            for key, value in img.info.items():
                metadata[f"Info_{key}"] = str(value)

        # Get PNG text chunks (tEXt, zTXt, iTXt)
        if hasattr(img, 'pnginfo') and img.pnginfo is not None:
            for key, value in img.pnginfo.items():
                metadata[f"PNG_{key}"] = str(value)

        # Format metadata for display
        output = "Image Metadata:\n\n"
        for key, value in metadata.items():
            output += f"{key}: {value}\n"

        return output
    except Exception as e:
        return f"Error processing image: {str(e)}"

In [ ]:
#@title Run Gradio interface
iface = gr.Interface(
    fn=get_image_metadata,
    inputs=[
        gr.Image(type="filepath", label="Upload PNG Image"),
        gr.Textbox(label="Or Enter Image URL", placeholder="https://example.com/image.png")
    ],
    outputs=gr.Textbox(label="Image Metadata"),
    title="PNG Image Metadata Extractor",
    description="Upload a PNG image or provide an image URL to view all its metadata"
)

# Launch the interface
iface.launch()